# GridSearch & Pipelines
GridSearch es una herramienta de optimización que usamos cuando ajustamos hiperparámetros. Definimos la cuadrícula(grid) de parámetros que queremos buscar y seleccionamos la mejor combinación de parámetros para nuestros datos.


## Método 1
Itera un único algoritmo sobre un conjunto de hiperparámetros, mediante la validación cruzada, iterando con el dataset dividido en train y val para recoger los errores y evaluar la mejor métrica. 

In [1]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()

parameters = {
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
    'degree': [1,2,3,4,5,6,7],
    'gamma': ['scale', 'auto']
}

svc = svm.SVC()

clf = GridSearchCV(estimator = svc,
                  param_grid = parameters,
                  n_jobs = -1,
                  cv = 10,
                  scoring="accuracy")

clf.fit(iris.data, iris.target)

,estimator,SVC()
,param_grid,"{'C': [0.001, 0.1, ...], 'degree': [1, 2, ...], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf', ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,0.1


In [3]:
clf.best_estimator_

,C,0.1
,kernel,'poly'
,degree,2
,gamma,'auto'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [4]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 0.1, 'degree': 2, 'gamma': 'auto', 'kernel': 'poly'}
0.9866666666666667


In [5]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(C=0.1, degree=2, gamma='auto', kernel='poly')
scores = cross_val_score(clf, iris.data, iris.target, cv=10)
scores

array([1.        , 0.93333333, 1.        , 1.        , 1.        ,
       1.        , 0.93333333, 1.        , 1.        , 1.        ])

In [6]:
import numpy as np
print(np.mean(scores))
print(np.std(scores))

0.9866666666666667
0.026666666666666658


## Método 2

Una forma más senior es montar un único gridsearch para iterar con varios modelos con otros hiperparámetros y con la validación cruzada.

In [7]:
import pickle

In [8]:
# Load libraries
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn import svm
# Set random seed
np.random.seed(0)

In [9]:
# Load data
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=2)

In [11]:
pipe = Pipeline(steps=[("scaler", StandardScaler()),
    ('classifier', RandomForestClassifier())
])

logistic_params = {
    'classifier': [LogisticRegression(max_iter=1000, solver='liblinear'), LogisticRegression(max_iter=10, solver='liblinear')],
    'classifier__penalty': ['l1', 'l2']
}

random_forest_params = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'classifier': [RandomForestClassifier()],
    'classifier__max_depth': [2,3]
}

svm_param = {
    'classifier': [svm.SVC()], # kernel rbf (gaussiano)
    'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
}

search_space = [
    logistic_params,
    random_forest_params,
    svm_param
]

clf = GridSearchCV(estimator = pipe,
                  param_grid = search_space,
                  cv = 5,
                  n_jobs=-1)

clf.fit(X_train, y_train)

/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial

,estimator,Pipeline(step...lassifier())])
,param_grid,"[{'classifier': [LogisticRegre...r='liblinear'), LogisticRegre...r='liblinear')], 'classifier__penalty': ['l1', 'l2']}, {'classifier': [RandomForestClassifier()], 'classifier__max_depth': [2, 3], 'scaler': [StandardScaler(), MinMaxScaler()]}, ...]"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,feature_range,"(0, ...)"


In [12]:
print(clf.best_estimator_)
print(clf.best_score_)
print(clf.best_params_)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('classifier', RandomForestClassifier(max_depth=2))])
0.975
{'classifier': RandomForestClassifier(), 'classifier__max_depth': 2, 'scaler': MinMaxScaler()}


In [13]:
clf.best_estimator_.predict(X_test)

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 2, 1,
       2, 1, 1, 0, 0, 2, 0, 2])

In [14]:
clf.best_estimator_.score(X_test,y_test)

0.9666666666666667

## Método 3

Otro uso puede ser la construcción de pipelines (tuberías) específicos para cada tipo de modelo.

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest # basado en p-values
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [17]:
reg_log = Pipeline(steps = [
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("reglog", LogisticRegression())
])
reg_log_param = {
    "imputer__strategy": ['mean', 'median'],
#    "reglog__penalty": ['l1', 'l2'],
    "reglog__penalty": ['l2'],
    "reglog__C": np.logspace(0, 4, 10)
}

In [18]:
rand_forest = RandomForestClassifier()
rand_forest_param = {
    "n_estimators": [50, 100, 150],
    "max_features": [1,2,3]
}


svm = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("svm", SVC())
])


svm_param = {
    'selectkbest__k': [2, 3, 4],
    'svm__kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'svm__C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
    'svm__degree': [1,2,3,4],
    'svm__gamma': ['scale', 'auto']
}


gs_reg_log = GridSearchCV(reg_log,
                         reg_log_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

gs_rand_forest = GridSearchCV(rand_forest,
                         rand_forest_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

gs_svm = GridSearchCV(svm,
                         svm_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

grids = {"gs_reg_log": gs_reg_log,
        "gs_rand_forest": gs_rand_forest,
        "gs_svm": gs_svm}

In [19]:
from sklearn.model_selection import train_test_split 
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 9 candidates, totalling 90 fits
Fitting 10 folds for each of 672 candidates, totalling 6720 fits


In [21]:
print(gs_reg_log.best_score_)
print(gs_reg_log.best_params_)
print(gs_reg_log.best_estimator_)
print(gs_reg_log.best_estimator_['reglog'])

0.9583333333333334
{'imputer__strategy': 'mean', 'reglog__C': np.float64(7.742636826811269), 'reglog__penalty': 'l2'}
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('reglog',
                 LogisticRegression(C=np.float64(7.742636826811269)))])
LogisticRegression(C=np.float64(7.742636826811269))


In [22]:
print(gs_rand_forest.best_score_)
print(gs_rand_forest.best_params_)
print(gs_rand_forest.best_estimator_)

0.9333333333333332
{'max_features': 1, 'n_estimators': 100}
RandomForestClassifier(max_features=1)


In [23]:
print(gs_svm.best_score_)
print(gs_svm.best_params_)
print(gs_svm.best_estimator_)
print(gs_svm.best_estimator_['svm'])

0.9666666666666668
{'selectkbest__k': 4, 'svm__C': 5, 'svm__degree': 1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=4)),
                ('svm', SVC(C=5, degree=1, kernel='linear'))])
SVC(C=5, degree=1, kernel='linear')


In [24]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)
best_grids

,Grid,Best score
2,gs_svm,0.966667
0,gs_reg_log,0.958333
1,gs_rand_forest,0.933333


In [25]:
gs_svm.best_estimator_

,steps,"[('scaler', ...), ('selectkbest', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,score_func,<function f_c...x73ae523916c0>
,k,4
,C,5
,kernel,'linear'


In [26]:
preds = gs_svm.best_estimator_.predict(X_test)
accuracy_score(y_test, preds)

0.9666666666666667

In [27]:
gs_reg_log.best_estimator_

,steps,"[('imputer', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [28]:
preds = gs_reg_log.best_estimator_.predict(X_test)
accuracy_score(y_test, preds)

1.0

In [29]:
preds = gs_rand_forest.best_estimator_.predict(X_test)
accuracy_score(y_test, preds)

1.0

 Tanto la regresión logística (pipeline) como el random forest son los modelos que mejor generalizan

In [30]:
gs_svm.best_estimator_

,steps,"[('scaler', ...), ('selectkbest', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,score_func,<function f_c...x73ae523916c0>
,k,4
,C,5
,kernel,'linear'


In [31]:
gs_svm.best_estimator_['svm']

,C,5
,kernel,'linear'
,degree,1
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [32]:
preds = gs_svm.best_estimator_.predict(X_test)
accuracy_score(y_test, preds)

0.9666666666666667

In [33]:
gs_svm.best_estimator_['svm']

,C,5
,kernel,'linear'
,degree,1
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [34]:
# El mejor modelo ha sido
best_model = gs_reg_log.best_estimator_
best_model.score(X_test, y_test)

1.0

In [35]:
gs_reg_log.best_params_

{'imputer__strategy': 'mean',
 'reglog__C': np.float64(7.742636826811269),
 'reglog__penalty': 'l2'}

In [36]:
# El mejor modelo ha sido
best_model = gs_reg_log.best_estimator_
best_model.score(X_test, y_test)

1.0

In [37]:
gs_reg_log.best_estimator_

,steps,"[('imputer', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


En train cv el reg_log mejor que random forest. Además es el más sencillo.

In [38]:
# El mejor modelo ha sido
best_model = gs_reg_log.best_estimator_
best_model.score(X_test, y_test)

1.0

In [39]:
gs_reg_log.best_estimator_

,steps,"[('imputer', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [40]:
import pickle

filename = 'finished_model'

with open(filename, 'wb') as archivo_salida:
    pickle.dump(best_model, archivo_salida)

In [41]:
with open(filename, 'rb') as archivo_entrada:
    modelo_importado = pickle.load(archivo_entrada)

In [42]:
modelo_importado.score(X_test, y_test)*100

100.0

In [43]:
modelo_importado.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [44]:
modelo_importado

,steps,"[('imputer', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [ ]:
# modelo_importado.predict(X_new)

Ya hemos escogido modelo gracias a los datos de validación. Ahora habría que entrenar el modelo con TODOS los datos de train.

## RandomSearch
El problema que tiene el GridSearchCV es que computacionalmente es muy costoso cuando el espacio dimensional de los hiperparámetros es grande.

Mediante el RandomSearch no se prueban todas las combinaciones, sino unas cuantas de manera aleatoria. Funciona bien con datasets con pocas features. Incluso [hay papers](https://www.jmlr.org/papers/v13/bergstra12a.html) que aseguran que es más eficiente RandomSearch frente a GridSearch


In [45]:
np.logspace(0, 4, 10)

array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04])

In [46]:
from sklearn.model_selection import RandomizedSearchCV

reg_log = Pipeline(steps=[
                          ("imputer",SimpleImputer()),
                          ("scaler",StandardScaler()),
                          ("reglog",LogisticRegression())
                         ])

reg_log_param = {    
                 "imputer__strategy": ['mean', 'median', 'most_frequent'],
#                 "reglog__penalty": ["l1","l2"], 
                 "reglog__penalty": ["l2"], 
                 "reglog__C": np.logspace(0, 4, 10)
                }


search = RandomizedSearchCV(reg_log,
                           reg_log_param,
                           n_iter = 50,
                           scoring='accuracy',
                           n_jobs=-1,
                           cv=10)

# execute search
result = search.fit(X_train, y_train)

# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
print('Best Estimator: %s' % result.best_estimator_)

/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/.venv/lib/python3.12/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 30 is smaller than n_iter=50. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Score: 0.9583333333333334
Best Hyperparameters: {'reglog__penalty': 'l2', 'reglog__C': np.float64(7.742636826811269), 'imputer__strategy': 'mean'}
Best Estimator: Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('reglog',
                 LogisticRegression(C=np.float64(7.742636826811269)))])


In [47]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
print('Best Estimator: %s' % result.best_estimator_)

Best Score: 0.9583333333333334
Best Hyperparameters: {'reglog__penalty': 'l2', 'reglog__C': np.float64(7.742636826811269), 'imputer__strategy': 'mean'}
Best Estimator: Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('reglog',
                 LogisticRegression(C=np.float64(7.742636826811269)))])


### Otros Steps

In [48]:
df = pd.read_csv('data/titanic.csv')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [50]:
df = df[['survived', 'age', 'fare', 'class', 'embark_town']]
df.head()

,survived,age,fare,class,embark_town
0,0,22.0,7.2500,Third,Southampton
1,1,38.0,71.2833,First,Cherbourg
2,1,26.0,7.9250,Third,Southampton
3,1,35.0,53.1000,First,Southampton
4,0,35.0,8.0500,Third,Southampton


In [51]:
X = df.drop('survived', axis=1)
y = df['survived']

In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


column_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'fare']),
        ('cat', OneHotEncoder(), ['class', 'embark_town'])
    ]
)

X_transformado = column_transformer.fit_transform(X)

In [53]:
X_transformado

array([[-0.53037664, -0.50244517,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.57183099,  0.78684529,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.25482473, -0.48885426,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [        nan, -0.17626324,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.25482473, -0.04438104,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.15850313, -0.49237783,  0.        , ...,  1.        ,
         0.        ,  0.        ]], shape=(891, 9))

In [54]:
X_new = pd.DataFrame(X_transformado)
X_new.columns = column_transformer.get_feature_names_out()
X_new.head()

,num__age,num__fare,cat__class_First,cat__class_Second,cat__class_Third,cat__embark_town_Cherbourg,cat__embark_town_Queenstown,cat__embark_town_Southampton,cat__embark_town_nan
0,-0.530377,-0.502445,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.571831,0.786845,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.254825,-0.488854,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.365167,0.420730,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.365167,-0.486337,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('log', FunctionTransformer(np.log1p), ['columna_a_transformar']),

    ]
)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Definir preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'fare']),
        ('cat', OneHotEncoder(), ['class', 'embark_town'])
    ]
)

# Crear el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Predecir
y_pred = pipeline.predict(X_test)

In [58]:
y_pred

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1])

# Análisis exploratorio automatizado

In [59]:
import pandas as pd

In [60]:
from sklearn import datasets

In [61]:
iris = datasets.load_iris()

In [62]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
# %pip install ydata-profiling setuptools

In [63]:
from ydata_profiling import ProfileReport

/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [64]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("your_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 348.89it/s]
